In [ ]:
%matplotlib inline
import skimage.io
import skimage.morphology
import matplotlib.pyplot as plt

import os.path
import os

import utils.metrics
import utils.model_builder

import numpy as np

import tensorflow as tf
import keras

In [ ]:
debug = True

cell_min_size = 16

boundary_boost_factor = 1

In [ ]:
base_dir = "/data1/image-segmentation/BBBC022/unet/"

dir_images = base_dir + "split/validation/x/0/"

tag = "01"

out_dir_probmap = base_dir + "experiments/" + tag + "/out/prob/"
out_dir_label = base_dir + "experiments/" + tag + "/out/segm/"

os.makedirs(out_dir_probmap, exist_ok=True)
os.makedirs(out_dir_label, exist_ok=True)

weights_path = base_dir + "experiments/" + tag + "/checkpoints/0015.hdf5"

In [ ]:
# Use the following configuration if you want to test on CPUs
# os.environ['CUDA_VISIBLE_DEVICES'] = ''
# configuration = tf.ConfigProto(
#       intra_op_parallelism_threads=1,
#       inter_op_parallelism_threads=1)

# Configuration to run on GPU
configuration = tf.ConfigProto()
configuration.gpu_options.allow_growth = True
configuration.gpu_options.visible_device_list = "0"

session = tf.Session(config = configuration)

# apply session
keras.backend.set_session(session)

In [ ]:
batch_size = 1

bit_depth = 8

imagebuffer = skimage.io.imread_collection(dir_images + '*.png')
images = imagebuffer.concatenate()

dim1 = images.shape[1]
dim2 = images.shape[2]

images = images.reshape((-1, dim1, dim2, 1))

# preprocess (assuming images are encoded as 8-bits in the preprocessing step)
images = images / 255

# build model and laod weights
model = utils.model_builder.get_model_3_class(dim1, dim2)
model.load_weights(weights_path)

# Use the following line to measure single image testing time
#%time model.predict(images[0:1,...], batch_size = 1)

# Normal prediction time
predictions = model.predict(images, batch_size = 1)

print(predictions.shape)

model.summary()

In [ ]:
# print one prediction for debug purposes
if(debug):
    plt.imshow(predictions[42].squeeze())
    plt.show()
    print(np.max(predictions[42][:,:,1]))

In [ ]:
for i in range(len(images)):

    filename = imagebuffer.files[i]
    filename = os.path.basename(filename)
    print(filename)
    
    probmap = predictions[i].squeeze() #[:,:,2]
    
    plt.imshow(probmap)
    plt.show()
    
    skimage.io.imsave(out_dir_probmap + filename, probmap)
    
    pred = utils.metrics.probmap_to_pred(probmap, boundary_boost_factor)

    plt.imshow(pred)
    plt.show()
    
    label = utils.metrics.pred_to_label(pred, cell_min_size)
    
    plt.imshow(label)
    plt.show()
    
    skimage.io.imsave(out_dir_label + filename, label)